<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


##Resolución

### Alumna: Maria Fabiana Cid

Objetivo: utilizar la notebook dada en clase para el libro "Las Mil y una Noches" extraido  de textos.info

Se explora la utilización de SimpleRNN.

Librerías:

In [ ]:
import random
import io
import pickle

import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

### Datos
Utilizaremos como dataset el libro Las Mil y una Noches extraido en pdf de textos.info

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install PyMuPDF

import fitz  # PyMuPDF

pdf_path = "/content/drive/MyDrive/Anonimo - Las Mil y Una Noches.pdf"

doc = fitz.open(pdf_path)
corpus_text = ""

for page in doc:
    corpus_text += page.get_text()

print(corpus_text[:1000])  # Ver los primeros caracteres


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 103.0 MB/s eta 0:00:00
1
Las Mil y Una Noches
Anónimo
textos.info
Libros gratis - biblioteca digital abierta
2
Texto núm. 4140
Título: Las Mil y Una Noches
Autor: Anónimo
Etiquetas: Cuento
Editor: Edu Robsy
Fecha de creación: 24 de diciembre de 2018
Fecha de modificación: 27 de diciembre de 2018
Edita textos.info
Maison Carrée
c/ Ramal, 48
07730 Alayor - Menorca
Islas Baleares
España
Más textos disponibles en http://www.textos.info
3
Una palabra del traductor a sus amigos
Yo ofrezco
desnudas, vírgenes, intactas y sencillas,
para mis delicias y el placer de mis amigos,
estas noches árabes vividas, soñadas y traducidas sobre su tierra natal y 
sobre el agua
Ellas me fueron dulces durante los ocios en remotos mares, bajo un cielo 
ahora lejano.
Por eso las doy.
Sencillas, sonrientes y llenas de ingenuidad, como la musulmana 
Schehrazada, su madre suculenta que las dió a luz en el misterio; 
fermentando con emoción en los brazos de u

### Elegir el tamaño del contexto

En este caso, como el modelo de lenguaje es por caracteres, todo un gran corpus
de texto puede ser considerado un documento en sí mismo y el tamaño de contexto
puede ser elegido con más libertad en comparación a un modelo de lenguaje tokenizado por palabras y dividido en documentos más acotados.

In [ ]:
# Preprocesar: minúsculas, eliminar caracteres raros
corpus_text = corpus_text.lower()
corpus_text = re.sub(r'[^a-záéíóúüñ\s]', '', corpus_text)


In [ ]:
# seleccionamos el tamaño de contexto
max_context_size = 100

In [ ]:
# en este caso el vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(corpus_text)

In [ ]:
# la longitud de vocabulario de caracteres es:
len(chars_vocab)

35

In [ ]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

###  Tokenizar

In [ ]:
# tokenizamos el texto completo
tokenized_text = [char2idx[ch] for ch in corpus_text]

In [ ]:
tokenized_text[:1000]

[10,
 6,
 8,
 23,
 34,
 0,
 17,
 6,
 34,
 3,
 34,
 4,
 25,
 8,
 34,
 25,
 9,
 20,
 24,
 5,
 23,
 10,
 8,
 25,
 31,
 25,
 17,
 0,
 9,
 10,
 29,
 5,
 12,
 29,
 9,
 23,
 17,
 25,
 22,
 9,
 10,
 6,
 17,
 30,
 19,
 9,
 23,
 34,
 26,
 19,
 8,
 29,
 17,
 23,
 34,
 34,
 30,
 17,
 30,
 6,
 17,
 9,
 29,
 5,
 20,
 8,
 34,
 32,
 17,
 26,
 17,
 29,
 8,
 6,
 34,
 8,
 30,
 17,
 5,
 19,
 29,
 8,
 10,
 10,
 29,
 5,
 12,
 29,
 9,
 34,
 25,
 21,
 0,
 34,
 10,
 29,
 16,
 29,
 4,
 6,
 9,
 34,
 6,
 8,
 23,
 34,
 0,
 17,
 6,
 34,
 3,
 34,
 4,
 25,
 8,
 34,
 25,
 9,
 20,
 24,
 5,
 23,
 10,
 8,
 4,
 29,
 9,
 19,
 34,
 8,
 25,
 31,
 25,
 17,
 0,
 9,
 10,
 5,
 29,
 17,
 7,
 4,
 5,
 29,
 8,
 23,
 34,
 20,
 4,
 5,
 25,
 29,
 9,
 10,
 5,
 32,
 17,
 29,
 9,
 19,
 34,
 5,
 32,
 4,
 34,
 19,
 9,
 30,
 23,
 3,
 10,
 22,
 5,
 20,
 24,
 8,
 34,
 32,
 5,
 34,
 20,
 19,
 5,
 8,
 20,
 17,
 31,
 25,
 34,
 34,
 32,
 5,
 34,
 32,
 17,
 20,
 17,
 5,
 0,
 30,
 19,
 5,
 34,
 32,
 5,
 34,
 10,
 22,
 5,
 20,
 24,
 8,
 34,
 32,
 5,


### Organizando y estructurando el dataset

In [ ]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

In [ ]:
# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

In [ ]:
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]

In [ ]:
tokenized_sentences_train = [train_text[init:init+max_context_size] for init in range(len(train_text)-max_context_size+1)]

In [ ]:
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

In [ ]:
X.shape

(6017806, 100)

In [ ]:
X[0,:10]

array([10,  6,  8, 23, 34,  0, 17,  6, 34,  3])

In [ ]:
y[0,:10]

array([ 6,  8, 23, 34,  0, 17,  6, 34,  3, 34])

In [ ]:
vocab_size = len(chars_vocab)

# Definiendo el modelo

In [ ]:
from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense
from keras.models import Model, Sequential

In [ ]:
model = Sequential()

model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, None, 35)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, None, 200)      │        47,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 35)       │         7,035 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,235 (211.86 KB)

 Trainable params: 54,235 (211.86 KB)

 Non-trainable params: 0 (0.00 B)


### Definir el modelo

In [ ]:
class PerplexityCallback(keras.callbacks.Callback):
    def __init__(self, validation_data=None):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        # Calculate and log training perplexity
        train_loss = logs.get('loss')
        if train_loss is not None:
            train_perplexity = np.exp(train_loss)
            print(f'\nTraining Perplexity: {train_perplexity:.4f}')
            logs['perplexity'] = train_perplexity

        # Calculate validation perplexity if validation data exists
        if self.validation_data is not None:
            val_loss = logs.get('val_loss')
            if val_loss is not None:
                val_perplexity = np.exp(val_loss)
                print(f'Validation Perplexity: {val_perplexity:.4f}')
                logs['val_perplexity'] = val_perplexity


### Entrenamiento

In [ ]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, mientras más grande mejor.
hist = model.fit(X, y, epochs=20, callbacks=[PerplexityCallback(tokenized_sentences_val)],  batch_size=300)

Epoch 1/20
20060/20060 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8776
Training Perplexity: 5.7987
20060/20060 ━━━━━━━━━━━━━━━━━━━━ 236s 11ms/step - loss: 1.8776 - perplexity: 5.7987
Epoch 2/20
20059/20060 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.6752
Training Perplexity: 5.3115
20060/20060 ━━━━━━━━━━━━━━━━━━━━ 211s 10ms/step - loss: 1.6752 - perplexity: 5.3115
Epoch 3/20
20056/20060 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.6587
Training Perplexity: 5.2399
20060/20060 ━━━━━━━━━━━━━━━━━━━━ 209s 10ms/step - loss: 1.6587 - perplexity: 5.2399
Epoch 4/20
20056/20060 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.6508
Training Perplexity: 5.2034
20060/20060 ━━━━━━━━━━━━━━━━━━━━ 210s 10ms/step - loss: 1.6508 - perplexity: 5.2034
Epoch 5/20
20056/20060 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.6458
Training Perplexity: 5.1803
20060/20060 ━━━━━━━━━━━━━━━━━━━━ 212s 11ms/step - loss: 1.6458 - perplexity: 5.1803
Epoch 6/20
20056/20060 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.6423
Training

In [ ]:
model.save('my_model_3.keras')


In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/my_model_3.keras')


### Predicción del próximo caracter

In [ ]:
# Se puede usar gradio para probar el modelo
# Gradio es una herramienta muy útil para crear interfaces para ensayar modelos
# https://gradio.app/

!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
import gradio as gr

def model_response(human_text):

    # Encodeamos
    encoded = [char2idx[ch] for ch in human_text.lower() ]
    # Si tienen distinto largo
    encoded = pad_sequences([encoded], maxlen=max_context_size, padding='pre')

    # Predicción softmax
    y_hat = np.argmax(model.predict(encoded)[0,-1,:])


    # Debemos buscar en el vocabulario el caracter
    # que corresopnde al indice (y_hat) predicho por le modelo
    out_word = ''
    out_word = idx2char[y_hat]

    # Agrego la palabra a la frase predicha
    return human_text + out_word

iface = gr.Interface(
    fn=model_response,
    inputs=["textbox"],
    outputs="text")

iface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ed36db241d96e6bc9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ed36db241d96e6bc9e.gradio.live


### Generación de secuencias

In [ ]:
def generate_seq(model, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de caracteres a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	# generate a fixed number of words
    for _ in range(n_words):
		# Encodeamos
        encoded = [char2idx[ch] for ch in output_text.lower() ]
		# Si tienen distinto largo
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		# Predicción softmax
        y_hat = np.argmax(model.predict(encoded,verbose=0)[0,-1,:])
		# Vamos concatenando las predicciones
        out_word = ''

        out_word = idx2char[y_hat]

		# Agrego las palabras a la frase predicha
        output_text += out_word
    return output_text

In [ ]:
input_text='habia una vez'

generate_seq(model, input_text, max_length=max_context_size, n_words=30)

'habia una vez y el califa y el califa y el '

###  Beam search y muestreo aleatorio

In [ ]:
# funcionalidades para hacer encoding y decoding

def encode(text,max_length=max_context_size):

    encoded = [char2idx[ch] for ch in text]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return ''.join([idx2char[ch] for ch in seq])

In [ ]:
from scipy.special import softmax

# función que selecciona candidatos para el beam search
def select_candidates(pred,num_beams,vocab_size,history_probs,history_tokens,temp,mode):

  # colectar todas las probabilidades para la siguiente búsqueda
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    idx_select = np.argsort(pred_large)[::-1][:num_beams] # beam search determinista
  elif mode == 'sto':
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams, p=softmax(pred_large/temp)) # beam search con muestreo aleatorio
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  # devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model,num_beams,num_words,input,temp=1,mode='det'):

    # first iteration

    # encode
    encoded = encode(input)

    # first prediction
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]

    # get vocabulary size
    vocab_size = y_hat.shape[0]

    # initialize history
    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    # select num_beams candidates
    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    # beam search loop
    for i in range(num_words-1):

      preds = []

      for hist in history_tokens:

        # actualizar secuencia de tokens
        input_update = np.array([hist[i+1:]]).copy()

        # predicción
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]

        preds.append(y_hat)

      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens[:,-(len(input)+num_words):]

In [ ]:
# predicción con beam search
salidas = beam_search(model,num_beams=10,num_words=20,input="habia una vez")

In [ ]:
salidas[0]

array([24,  8, 30, 17,  8, 34,  4, 25,  8, 34, 15,  5, 28, 34,  7,  4,  5,
       34, 23,  5, 34, 20,  8,  6,  6, 31, 34, 32, 17, 23, 20, 19,  5])

In [ ]:
# veamos las salidas
decode(salidas[0])

'habia una vez que se calló discre'

El modelo entiende estructuras narrativas, pero la generación fue interrumpida o desestabilizada por el muestreo. Vamos a probar una segunda opción con uso de LSTM.